# Cifar10 Dataset 모델링

- cifar10 오픈데이터셋 모델링.
 - Bulit-In Dataset
 - color image 10개 분류 32*32


1. Dataset Dataloader
2. augmentation 적용
3. modeling transfer learning
4. vgg19 이외의 모델들도 사용
5. transfer learning - finetuning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torchinfo import summary
from torch.utils.data import DataLoader

from module.train import fit
from module.utils import plot_fit_result

import os

device = 'cuda' if torch.cuda.is_available() else "cpu"

In [19]:
lr = 0.001
batch_size = 64
epochs = 1

In [18]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.5, hue=0.15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=180),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [30]:
datasets?

Type:        module
String form: <module 'torchvision.datasets' from 'c:\\classes\\PLAYDATA-DA-35\\classes\\env\\Lib\\site-packages\\torchvision\\datasets\\__init__.py'>
File:        c:\classes\playdata-da-35\classes\env\lib\site-packages\torchvision\datasets\__init__.py
Docstring:   <no docstring>

In [45]:
os.makedirs(r"C:\classes\deeplearning\datasets", exist_ok=True)
os.makedirs(r"C:\classes\deeplearning\saved_model\cifar10", exist_ok=True)
cifar10_train = datasets.CIFAR10(r'C:\classes\deeplearning\datasets', train=True, download=True, transform=train_transform)
cifar10_train, cifar10_valid = torch.utils.data.random_split(cifar10_train, [0.8, 0.2])
cifar10_test = datasets.CIFAR10(r'C:\classes\deeplearning\datasets', train=False, download=True, transform=test_transform)


Files already downloaded and verified
Files already downloaded and verified


In [46]:
cifar10_valid.transform = test_transform

In [48]:
train_loader = DataLoader(cifar10_train, batch_size, shuffle=True, drop_last=True,num_workers=os.cpu_count())
valid_loader = DataLoader(cifar10_valid, batch_size, num_workers=os.cpu_count())
test_loader = DataLoader(cifar10_test, batch_size, num_workers=os.cpu_count())

In [55]:
model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)

In [56]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
summary(model, (64,3,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [64, 1000]                --
├─Conv2d: 1-1                            [64, 64, 16, 16]          9,408
├─BatchNorm2d: 1-2                       [64, 64, 16, 16]          128
├─ReLU: 1-3                              [64, 64, 16, 16]          --
├─MaxPool2d: 1-4                         [64, 64, 8, 8]            --
├─Sequential: 1-5                        [64, 256, 8, 8]           --
│    └─Bottleneck: 2-1                   [64, 256, 8, 8]           --
│    │    └─Conv2d: 3-1                  [64, 64, 8, 8]            4,096
│    │    └─BatchNorm2d: 3-2             [64, 64, 8, 8]            128
│    │    └─ReLU: 3-3                    [64, 64, 8, 8]            --
│    │    └─Conv2d: 3-4                  [64, 64, 8, 8]            36,864
│    │    └─BatchNorm2d: 3-5             [64, 64, 8, 8]            128
│    │    └─ReLU: 3-6                    [64, 64, 8, 8]            --
│ 

In [24]:
for param in model.parameters():
    param.requires_grad = False

In [25]:
model.fc = nn.Linear(2048, 10)

In [54]:
save_path = 'saved_models/resnet_cifar10.pth'
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

fit(train_loader, valid_loader, model, loss_fn, optimizer, epochs, save_best_model=True, save_model_path=save_path, 
    early_stopping=True, patience=3, device='cpu', mode='multi')

KeyboardInterrupt: 

In [53]:
summary(model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └─C